In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# multivariate output multi-step 1d cnn
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import date,datetime
import itertools
import statsmodels.api as sm
import matplotlib
from statsmodels.graphics.tsaplots import (plot_acf, plot_pacf, month_plot,
                                           quarter_plot, seasonal_plot)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Datasets:')
os.listdir('../input/acea-water-prediction')

In [ ]:
#let's get each dataset
Aquifer_Doganella = pd.read_csv('../input/acea-water-prediction/Aquifer_Doganella.csv', index_col='Date')
Aquifer_Auser = pd.read_csv('../input/acea-water-prediction/Aquifer_Auser.csv', index_col='Date')
Water_Spring_Amiata = pd.read_csv('../input/acea-water-prediction/Water_Spring_Amiata.csv', index_col='Date')
Lake_Bilancino = pd.read_csv('../input/acea-water-prediction/Lake_Bilancino.csv', index_col='Date')
Water_Spring_Madonna_di_Canneto = pd.read_csv('../input/acea-water-prediction/Water_Spring_Madonna_di_Canneto.csv', index_col='Date')
Aquifer_Luco = pd.read_csv('../input/acea-water-prediction/Aquifer_Luco.csv',index_col='Date')
Aquifer_Petrignano = pd.read_csv('../input/acea-water-prediction/Aquifer_Petrignano.csv', index_col='Date')
Water_Spring_Lupa = pd.read_csv('../input/acea-water-prediction/Water_Spring_Lupa.csv', index_col='Date')
River_Arno = pd.read_csv('../input/acea-water-prediction/River_Arno.csv', index_col='Date')

In [ ]:
datasets = [Aquifer_Doganella,Aquifer_Auser,Water_Spring_Amiata,Lake_Bilancino,Water_Spring_Madonna_di_Canneto,
 Aquifer_Luco,Aquifer_Petrignano,Water_Spring_Lupa,River_Arno]
dataset_names = ['Aquifer_Doganella',
 'Aquifer_Auser',
 'Water_Spring_Amiata',
 'Lake_Bilancino',
 'Water_Spring_Madonna_di_Canneto',
 'Aquifer_Luco',
 'Aquifer_Petrignano',
 'Water_Spring_Lupa',
 'River_Arno']

In [ ]:
for i in range(len(datasets)):
    print('{} :\n{}\nShape:{}'.format(dataset_names[i],datasets[i].dtypes.value_counts(),datasets[i].shape))
    print('='*30)

In [ ]:
df_R = pd.read_csv('../input/acea-water-prediction/River_Arno.csv')
df_R['Date'] = pd.to_datetime(df_R.Date, format = '%d/%m/%Y')
#df_R

In [ ]:
df_R.info()

In [ ]:
df_R['day'] = df_R['Date'].dt.day
df_R['month'] = df_R['Date'].dt.month
df_R['year'] = df_R['Date'].dt.year
years = df_R['year'].unique()
#df_R

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)
sns.boxplot(x='year', y='Hydrometry_Nave_di_Rosano', data=df_R, ax=axes[0])
sns.boxplot(x='month', y='Hydrometry_Nave_di_Rosano', data=df_R.loc[~df_R.year.isin([1998, 2020]), :])

# Set Title
axes[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize=18);
axes[0].tick_params(axis='x', labelrotation = 45)
axes[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize=18)
#plt.xticks(rotation = 90)
plt.show()

In [ ]:
fig = plt.subplots(figsize=(16,7))
df_R['Hydrometry_Nave_di_Rosano'].replace(0, np.nan, inplace=True)
sns.lineplot(x='Date',y='Hydrometry_Nave_di_Rosano',data=df_R)
#fig.update_xaxes(rangeslider_visible=True)
plt.show()

In [ ]:
features_to_fill = ['Date', 'Rainfall_Le_Croci', 'Rainfall_Cavallina', 'Rainfall_S_Agata',
       'Rainfall_Mangona', 'Rainfall_S_Piero', 'Rainfall_Vernio',
       'Rainfall_Stia', 'Rainfall_Consuma', 'Rainfall_Incisa',
       'Rainfall_Montevarchi', 'Rainfall_S_Savino', 'Rainfall_Laterina',
       'Rainfall_Bibbiena', 'Rainfall_Camaldoli', 'Temperature_Firenze',
       'Hydrometry_Nave_di_Rosano']

In [ ]:
temp_df = df_R[features_to_fill].copy()

In [ ]:
temp_df['year'] = pd.DatetimeIndex(temp_df['Date']).year 
temp_df['month'] = pd.DatetimeIndex(temp_df['Date']).month 


month_in_year = 12
temp_df['month_sin'] = np.sin(2*np.pi*temp_df.month/month_in_year)
temp_df['month_cos'] = np.cos(2*np.pi*temp_df.month/month_in_year)


temp_df['season'] = temp_df.month%12 // 3 + 1

temp_df['day_of_year'] = pd.DatetimeIndex(temp_df['Date']).dayofyear

days_in_year = 365.25
temp_df['day_of_year_sin'] = np.sin(2*np.pi*temp_df.day_of_year/days_in_year)
temp_df['day_of_year_cos'] = np.cos(2*np.pi*temp_df.day_of_year/days_in_year)

temp_df['week_of_year'] = pd.DatetimeIndex(temp_df['Date']).weekofyear

weeks_in_year = 52.1429
temp_df['week_of_year_sin'] = np.sin(2*np.pi*temp_df.week_of_year/weeks_in_year)
temp_df['week_of_year_cos'] = np.cos(2*np.pi*temp_df.week_of_year/weeks_in_year)

In [ ]:
# Prepare training, validation, and test data sets

features = ['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'week_of_year_sin', 'week_of_year_cos', 'year', 'season']

for attribute in features_to_fill[1:]:
    print(attribute)
    target = attribute
    X = temp_df[temp_df[attribute].notna()][features]#.reset_index(drop=True)
    y = temp_df[temp_df[attribute].notna()][target]#.reset_index(drop=True)
    print(X.shape)
    print(y.shape)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    X_test = temp_df[features]#.reset_index(drop=True)
    print(X_test.shape)
    # Model
    params = {'num_leaves': 32,
              'objective': 'regression_l1',
              'max_depth': 8,
              'learning_rate': 0.05,
              "metric": 'mae',
              'seed' : 42
            }

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=False,  early_stopping_rounds=100)

    y_pred_valid = clf.predict(X_valid)

    old = temp_df[attribute].copy()
    y_pred = clf.predict(X_test)
    print(y_pred)
    df_R[attribute] = np.where(df_R[attribute].isna(), y_pred, df_R[attribute])

    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 3))

    sns.lineplot(x=df_R.Date, y=df_R[attribute], color='red',label='predicted')
    sns.lineplot(x=df_R.Date, y=old.fillna(np.inf), color='blue', label = 'original')
    ax.set_title('Fill NaN by Prediction', fontsize=14)
    ax.set_ylabel(ylabel=attribute, fontsize=14)
    ax.set_xlim([date(1998, 1, 1), date(2020, 6, 30)])
    plt.show()

In [ ]:
fig = plt.subplots(figsize=(16,7))
sns.lineplot(x=df_R.Date,y = df_R.Hydrometry_Nave_di_Rosano)
plt.show()

In [ ]:
y = df_R['Hydrometry_Nave_di_Rosano']
X = df_R.drop(['Hydrometry_Nave_di_Rosano','Date','month','year'],axis=1)
print(X)
print(y)

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
raw_seq = df_R['Hydrometry_Nave_di_Rosano']
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

In [ ]:
# univariate bidirectional lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
# fit model
history = model.fit(X, y, epochs=200, verbose=1)

In [ ]:
model.summary()

In [ ]:
# demonstrate prediction
x_input = array(X[:len(X)-50])
print(x_input)
x_input = x_input.reshape((len(X)-50, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
diff_results =[]
for i in range(0,len(yhat)):
    diff_results.append(y - yhat)

In [ ]:
diff_results

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(X, y, verbose=0)
_, test_acc = model.evaluate(X, y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

* Old code below------------------------------------------------>

In [ ]:
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]


In [ ]:
X_train.info()

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
raw_seq = array(df_R['Hydrometry_Nave_di_Rosano'])

In [ ]:
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
# split into train and test
n_train = int(X.shape[0]*0.8)
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

In [ ]:
# multi-step data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
n_steps_in, n_steps_out = 3, 2
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=1000, verbose=1)

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
X.shape

In [ ]:
y.shape[1]

In [ ]:
Y = y.reshape((y.shape[0], y.shape[1], n_features))

In [ ]:
Y.shape

In [ ]:

n_steps = 3

x_input = array(Y[0:300,0])
x_input = x_input.reshape((100 , n_steps, n_features))
yhat = model.predict(x_input, verbose=1)
print(yhat)

In [ ]:
y

In [ ]:
trainX = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
testX = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# demonstrate prediction
x_input = array(trainX)
x_input = x_input.reshape((X_train.shape[0], n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


In [ ]:
import math
from pandas import DataFrame
from pandas import Series
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime(x, '%d/%m/%Y')
 
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)
 
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	# transform data to be stationary
	diff_series = difference(raw_values, 1)
	diff_values = diff_series.values
	diff_values = diff_values.reshape(len(diff_values), 1)
	# rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	# transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	# split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test
 
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
	# reshape training into [samples, timesteps, features]
	X, y = train[:, 0:n_lag], train[:, n_lag:]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	# design network
	model = Sequential()
	model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(y.shape[1]))
	model.compile(loss='mean_squared_error', optimizer='adam',  metrics=['accuracy'])
	# fit network
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
		model.reset_states()
	return model
 
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
	# reshape input pattern to [samples, timesteps, features]
	X = X.reshape(1, 1, len(X))
	# make forecast
	forecast = model.predict(X, batch_size=n_batch)
	# convert to array
	return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
	forecasts = list()
	for i in range(len(test)):
		X, y = test[i, 0:n_lag], test[i, n_lag:]
		# make forecast
		forecast = forecast_lstm(model, X, n_batch)
		# store the forecast
		forecasts.append(forecast)
	return forecasts
 
# invert differenced forecast
def inverse_difference(last_ob, forecast):
	# invert first forecast
	inverted = list()
	inverted.append(forecast[0] + last_ob)
	# propagate difference forecast using inverted first value
	for i in range(1, len(forecast)):
		inverted.append(forecast[i] + inverted[i-1])
	return inverted
 
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
	inverted = list()
	for i in range(len(forecasts)):
		# create array from forecast
		forecast = array(forecasts[i])
		forecast = forecast.reshape(1, len(forecast))
		# invert scaling
		inv_scale = scaler.inverse_transform(forecast)
		inv_scale = inv_scale[0, :]
		# invert differencing
		index = len(series) - n_test + i - 1
		last_ob = series.values[index]
		inv_diff = inverse_difference(last_ob, inv_scale)
		# store
		inverted.append(inv_diff)
	return inverted
 
# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
	for i in range(n_seq):
		actual = [row[i] for row in test]
		predicted = [forecast[i] for forecast in forecasts]
		rmse = math.sqrt(mean_squared_error(actual, predicted))
		print('t+%d RMSE: %f' % ((i+1), rmse))
 
# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
	# plot the entire dataset in blue
	
    pyplot.plot(series.values)
	# plot the forecasts in red
	for i in range(len(forecasts)):
		off_s = len(series) - n_test + i - 1
		off_e = off_s + len(forecasts[i]) + 1
		xaxis = [x for x in range(off_s, off_e)]
		yaxis = [series.values[off_s]] + forecasts[i]
		pyplot.plot(xaxis, yaxis, color='red')
	# show the plot
	pyplot.show()

In [ ]:
# load dataset
series = pd.read_csv('../input/acea-water-prediction/River_Arno.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
series.drop(series.iloc[:, 0:15], inplace = True, axis = 1)

# configure
n_lag = 1
n_seq = 3
n_test = 100
n_epochs = 100
n_batch = 1
n_neurons = 1

In [ ]:
# prepare data
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

In [ ]:
# make forecasts

forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
##actual = [row[n_lag:] for row in test]
##actual = inverse_transform(series, actual, scaler, n_test+2)
# evaluate forecasts
##evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# plot forecasts
fig = plt.subplots(figsize = (50,7))
plot_forecasts(series, forecasts, n_test+2)

In [ ]:
series